In [1]:
from transformers import AutoModelForCausalLM
from src.sae.model import SAE

model = AutoModelForCausalLM.from_pretrained("ExplosionNuclear/Llama-2.3-3B-Instruct-special")

p = next(model.parameters()); d = model.config.hidden_size
sae_attn = SAE(d, 4096).to(p.device, dtype=p.dtype)
sae_mlp  = SAE(d, 4096).to(p.device, dtype=p.dtype)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
model
saes = {"1_attn": SAE(d, 4096), "1_mlp": SAE(d, 4096)}

outputs = {"1_attn": [], "1_mlp": []}

def make_hook(sae, name, inject=False):
    def hook(mod, inp, out):
        sae_reconstructed = sae(out[0])
        outputs[name].append(sae_reconstructed)
        return None if not inject else sae_reconstructed
    return hook



In [43]:
model.model.layers[1].self_attn._forward_hooks.clear()
model.model.layers[1].mlp._forward_hooks.clear()

In [44]:
model.model.layers[1].self_attn.register_forward_hook(make_hook(saes["1_attn"], "1_attn"))
model.model.layers[1].mlp.register_forward_hook(make_hook(saes["1_mlp"], "1_mlp"))



In [45]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("ExplosionNuclear/Llama-2.3-3B-Instruct-special")

inputs = tokenizer("Hello, world!", return_tensors="pt")

model.eval()

out = model(**inputs)

In [46]:
outputs["1_attn"]

[tensor([[[ 0.0023, -0.0024, -0.0013,  ...,  0.0019, -0.0012, -0.0015],
          [ 0.0036, -0.0016, -0.0003,  ...,  0.0018, -0.0040, -0.0063],
          [-0.0053, -0.0032,  0.0012,  ..., -0.0007,  0.0003, -0.0040],
          [ 0.0015, -0.0059, -0.0095,  ...,  0.0061, -0.0023, -0.0040],
          [ 0.0063,  0.0005, -0.0009,  ...,  0.0039,  0.0006, -0.0039]]],
        grad_fn=<UnsafeViewBackward0>)]